In [17]:
!pip -q install langchain bitsandbytes accelerate transformers sentence_transformers unstructured pinecone-client numpy langchain_pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 6.4 MB/s eta 0:00:00


In [2]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch

In [3]:
URLs = ["https://www.datanetiix.com/"]

In [4]:
loader = UnstructuredURLLoader(urls = URLs)
data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [5]:
print(data)

[Document(page_content='+1 949-340-2540\n\ninfo@datanetiix.com\n\nLet\'s Talk\n\nGet your Dream Job\n\n\n\nHome\n\nAbout Us\n\nHistory\nLeadership\nAdvisors\nVision\nPartners\nWhy Datanetiix?\n\nClients\n\nServices\n\nApplication Development\nWeb/Desktop/Client Application\nContent Management System\nUser Experience Design\nMobility Services\nNative IOS and Android\nHybrid Application\nWearable App Development\nGaming and 3D Apps\nInnovative Technologies\nArtificial Intelligence(AI) and Machine Learning\nRobotic Process Automation(RPA)\nVirtual Reality and Augmented Reality\nBlockchain / NFT\'s\nInfrastructure Management\nAzure\nAWS\nCloud Services\nDevOps\nQuality Assurance\nPartner Services\n\nPlatforms\n\nSalesforce\nSales Cloud\nService Cloud\nMarketing Cloud\nCommerce Cloud\nCommunity / Experience Cloud\nPardot\nLightning\nApp Development\nHeroku\nAnalytics\nSalesforce Integration\nSalesforce Support\n Microsoft Dynamics 365 \nDynamics 365 for Finance and Operation\nDynamics 365 B

In [6]:
# chunkin
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [7]:
text_chunks=text_splitter.split_documents(data)

In [8]:
len(text_chunks)

39

In [9]:
# embedding
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

384

In [11]:
import os

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '4f72adde-3d6a-4327-a78c-0fbd481f35f1')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-east-1')

In [14]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [15]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [18]:
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore

In [19]:
vectorstore_from_texts = PineconeVectorStore.from_texts(
        [t.page_content for t in text_chunks],
        index_name='chatbot',
        embedding=embeddings,
    )


In [20]:
# LLM
notebook_login()

In [21]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [22]:
pipe = transformers.pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [23]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.7})

In [25]:
# initialize retrival QA
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore_from_texts.as_retriever())

In [51]:
query = "does datanettix provide PowerBI solutions"

In [52]:
print(qa.run(query))

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Learn More
Application Development
We are seasoned builders of custom applications your business growth requires. Applications are a major driver of business communication systems today. Datanetiix offers Cloud and Desktop development skills that will keep your business on the forefront of efficiency, inside and outside your company.
Learn More
Mobility Services
As a pioneer in the development of Mobile Applications we have a successful track record in building mobile applications across major platforms for both Native and Hybrid models. We are well versed with Cross-platform mobile development and integration.
Learn More
Blockchain

LEARN MORE
MICROSOFT DYNAMICS 365
BUSINESS CENTRAL
LEARN MORE
MICROSOFT DYNAMICS 365
LEARN MORE
MICROSOFT DYNAMICS 365
LEARN MORE
APPLICATION DEVELOPMENT
Learn More
MOBILITY SERVICES
Learn More
